<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/8_version_kappa_eklendi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import random
import os
import numpy as np

# --- AYARLAR ---
DOSYA_EGITIM = "veri.txt"
DOSYA_TEST = "test.txt"

# Biyolojik Alfabe
DNA_BASES = list("ACGT")
AMINO_ACIDS = list("ACDEFGHIKLMNPQRSTVWY")

# Hasta/Sağlam Oranı Ayarı (0.60 = %60 Hasta üretmeye çalışır)
HEDEF_HASTA_ORANI = 0.60

def rastgele_dizi(uzunluk, alfabe):
    return "".join(random.choice(alfabe) for _ in range(uzunluk))

def generate_pattern_sequence(length, vocab, pattern):
    """Motif içeren dizi oluşturur (Modelin yakalaması için ipucu)"""
    seq = list(rastgele_dizi(length, vocab))
    start = random.randint(0, length - len(pattern))
    for i, char in enumerate(pattern):
        seq[start + i] = char
    return "".join(seq)

def biyolojik_satir_olustur_dengeli():
    """
    Bu fonksiyon rastgele değer üretmek yerine, önce HASTA mı SAĞLAM mı olacağına
    karar verir, sonra ona uygun sayısal değerler üretir.
    Böylece formül yine tutarlı kalır ama istediğimiz kadar hasta verisi elde ederiz.
    """

    # Adım 1: Bu satırın ne olmasını istiyoruz? (Hedef Belirleme)
    if random.random() < HEDEF_HASTA_ORANI:
        niyet = "HASTA"
    else:
        niyet = "SAGLAM"

    # Adım 2: Niyete uygun aralıklardan sayı seç (Bias Injection)
    if niyet == "HASTA":
        # Hasta olması için yüksek risk faktörleri seçiyoruz
        # Formül > 0.60 çıkmalı
        risk = random.uniform(0.55, 1.00)    # Yüksek Risk
        maf = random.uniform(0.00, 0.15)     # Düşük MAF (Nadir)
        cons = random.uniform(6.0, 10.0)     # Yüksek Korunmuşluk
        hydro = random.uniform(-5.0, -1.0)   # Negatif
        polar = random.uniform(2.0, 5.0)     # Pozitif
        weight_base = random.uniform(50.0, 90.0)
    else:
        # Sağlam olması için düşük risk faktörleri
        risk = random.uniform(0.00, 0.50)    # Düşük Risk
        maf = random.uniform(0.20, 0.50)     # Yüksek MAF (Yaygın)
        cons = random.uniform(0.0, 5.0)      # Düşük Korunmuşluk
        hydro = random.uniform(-1.0, 5.0)
        polar = random.uniform(-5.0, 1.0)
        weight_base = random.uniform(10.0, 50.0)

    # Adım 3: Formülü Uygula (Matematiksel Tutarlılık İçin)
    # Değerleri biz seçtik ama etiket yine formülden çıkmalı ki model matematiği öğrensin.
    score = (risk * 0.50) + (cons/10 * 0.30) + ((0.5 - maf) * 0.1) + (abs(hydro)/5 * 0.1)

    # Hafif gürültü ekle
    noise = random.uniform(-0.02, 0.02)
    final_score = score + noise

    # Adım 4: Etiketleme ve Dizi Oluşturma
    if final_score > 0.60:
        etiket = 1 # HASTA
        # Hasta dizilerinde motif (pattern) olma ihtimali yüksek
        dna = generate_pattern_sequence(11, DNA_BASES, "CGT") if random.random() > 0.15 else rastgele_dizi(11, DNA_BASES)
        prot = generate_pattern_sequence(11, AMINO_ACIDS, "WW") if random.random() > 0.15 else rastgele_dizi(11, AMINO_ACIDS)
        weight = weight_base + 10
    else:
        etiket = 0 # SAĞLAM
        # Sağlamlarda motif çok nadir olsun
        dna = generate_pattern_sequence(11, DNA_BASES, "AAA") if random.random() > 0.9 else rastgele_dizi(11, DNA_BASES)
        prot = rastgele_dizi(11, AMINO_ACIDS)
        weight = weight_base - 10

    return [
        dna, prot,
        f"{risk:.4f}", f"{maf:.4f}", f"{cons:.4f}",
        f"{hydro:.4f}", f"{polar:.4f}", f"{weight:.4f}",
        str(etiket)
    ]

def dosya_uret_ve_kaydet(dosya_adi, toplam_adet):
    print(f"\n⚙️ {dosya_adi} hazırlanıyor... (Hedef: {toplam_adet} satır)")

    veri_listesi = []
    hasta_sayaci = 0

    for _ in range(toplam_adet):
        satir = biyolojik_satir_olustur_dengeli()
        if satir[-1] == "1": hasta_sayaci += 1
        veri_listesi.append(satir)

    # Karıştır (Model sırayı ezberlemesin)
    random.shuffle(veri_listesi)

    with open(dosya_adi, "w", encoding="utf-8") as f:
        for satir in veri_listesi:
            f.write(",".join(satir) + "\n")

    oran = (hasta_sayaci / toplam_adet) * 100
    print(f"✅ {dosya_adi} bitti.")
    print(f"   📊 İstatistik: {hasta_sayaci} HASTA (%{oran:.1f}) | {toplam_adet - hasta_sayaci} SAĞLAM")

if __name__ == "__main__":
    print("="*60)
    print("🧬 GÜNCELLENMİŞ HASTA ODAKLI VERİ ÜRETİCİ")
    print("   (Strateji: Verilerin ~%60'ı HASTA olacak şekilde ayarlandı)")
    print("="*60)

    try:
        n_train = int(input(">> Eğitim için kaç veri üretilsin? (Örn: 5000): "))
        n_test = int(input(">> Test için kaç veri üretilsin? (Örn: 1000): "))
    except:
        print("⚠️ Sayı girilmedi, varsayılanlar kullanılıyor (5000/1000).")
        n_train, n_test = 5000, 1000

    dosya_uret_ve_kaydet(DOSYA_EGITIM, n_train)
    dosya_uret_ve_kaydet(DOSYA_TEST, n_test)

🧬 GÜNCELLENMİŞ HASTA ODAKLI VERİ ÜRETİCİ
   (Strateji: Verilerin ~%60'ı HASTA olacak şekilde ayarlandı)
>> Eğitim için kaç veri üretilsin? (Örn: 5000): 100000
>> Test için kaç veri üretilsin? (Örn: 1000): 10000

⚙️ veri.txt hazırlanıyor... (Hedef: 100000 satır)
✅ veri.txt bitti.
   📊 İstatistik: 57211 HASTA (%57.2) | 42789 SAĞLAM

⚙️ test.txt hazırlanıyor... (Hedef: 10000 satır)
✅ test.txt bitti.
   📊 İstatistik: 5710 HASTA (%57.1) | 4290 SAĞLAM


In [21]:
import numpy as np
import pandas as pd
import pickle
import joblib
import xgboost as xgb
import lightgbm as lgb
import os
import tensorflow as tf
import random

# Keras / TensorFlow
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import Callback

# Scikit-Learn Modelleri
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report

# --- AYARLAR ---
MAX_EPOCH_LIMIT = 40
BATCH_SIZE = 32

# Rastgelelikleri sabitle
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

print(">> SİSTEM BAŞLATILIYOR (GÜNCELLENMİŞ EĞİTİM MODU)...")

# --- 1. DOSYA OKUMA VE HAZIRLAMA FONKSİYONLARI ---

def dosya_oku(dosya_yolu):
    """
    Belirtilen txt dosyasını okur.
    Beklenen Format: DNA,PROTEIN,Risk,Maf,Cons,Hydro,Polar,Weight,Label
    """
    if not os.path.exists(dosya_yolu):
        print(f"❌ HATA: '{dosya_yolu}' bulunamadı! Lütfen dosyayı oluşturun.")
        return [], [], [], []

    print(f">> '{dosya_yolu}' dosyası okunuyor...")

    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []

    with open(dosya_yolu, "r", encoding="utf-8") as f:
        satirlar = f.readlines()

    for i, satir in enumerate(satirlar):
        satir = satir.strip()
        if not satir: continue # Boş satırı atla

        # Virgülle ayır
        cols = satir.split(',')

        # Basit hata kontrolü (9 sütun olmalı: 2 dizi + 6 özellik + 1 etiket)
        if len(cols) != 9:
            print(f"⚠️ Uyarı: Satır {i+1} formatı bozuk, atlandı. Sütun sayısı: {len(cols)}")
            continue

        try:
            d = cols[0].strip().upper()
            p = cols[1].strip().upper()
            # Sayısal özellikleri çek
            feats = [float(x) for x in cols[2:8]]
            l = int(cols[8])

            dna_seqs.append(d)
            prot_seqs.append(p)
            bio_feats.append(feats)
            labels.append(l)
        except Exception as e:
            print(f"⚠️ Hata (Satır {i+1}): {e}")

    return dna_seqs, prot_seqs, np.array(bio_feats).astype('float32'), np.array(labels)

# --- 2. VERİLERİ YÜKLE ---

# Eğitim verisini oku (veri.txt)
print("\n--- EĞİTİM VERİSİ YÜKLENİYOR ---")
dna_train_raw, prot_train_raw, num_train_raw, y_train_raw = dosya_oku("veri.txt")

if len(y_train_raw) == 0:
    print("❌ Eğitim verisi boş! Program durduruluyor.")
    exit()

print(f"✅ 'veri.txt' başarıyla yüklendi. Toplam kayıt: {len(y_train_raw)}")

# Test verisini oku (test.txt)
print("\n--- TEST VERİSİ YÜKLENİYOR ---")
dna_test_raw, prot_test_raw, num_test_raw, y_test_raw = dosya_oku("test.txt")
print(f"✅ 'test.txt' başarıyla yüklendi. Toplam kayıt: {len(y_test_raw)}")

# --- 3. TOKENIZATION (Sadece Eğitim Verisine Göre Eğitilir) ---
print("\n>> Tokenizer eğitiliyor...")
dna_tok = Tokenizer(char_level=True)
dna_tok.fit_on_texts(dna_train_raw) # Sadece eğitim setini görsün

prot_tok = Tokenizer(char_level=True)
prot_tok.fit_on_texts(prot_train_raw)

# Hem Train hem Test verisini sayısal diziye çevir
# Train
X_dna_train_all = pad_sequences(dna_tok.texts_to_sequences(dna_train_raw), maxlen=11, padding='post')
X_prot_train_all = pad_sequences(prot_tok.texts_to_sequences(prot_train_raw), maxlen=11, padding='post')
X_num_train_all = num_train_raw
y_train_all = y_train_raw

# Test (Eğer test verisi varsa dönüştür, yoksa boş geç)
if len(y_test_raw) > 0:
    X_dna_test = pad_sequences(dna_tok.texts_to_sequences(dna_test_raw), maxlen=11, padding='post')
    X_prot_test = pad_sequences(prot_tok.texts_to_sequences(prot_test_raw), maxlen=11, padding='post')
    X_num_test = num_test_raw
    y_test = y_test_raw
else:
    print("⚠️ UYARI: test.txt boş veya bulunamadı. Final test yapılamayacak.")
    X_dna_test, X_prot_test, X_num_test, y_test = None, None, None, None


# --- 4. DATA SPLIT (STACKING İÇİN) ---
# Stacking yapabilmek için "veri.txt" içindeki veriyi Base Modeller ve Meta Model için ikiye bölüyoruz.
print("\n>> Eğitim verisi 'Base' ve 'Meta' eğitim setlerine ayrılıyor...")
X_dna_base, X_dna_meta, X_prot_base, X_prot_meta, X_num_base, X_num_meta, y_base, y_meta = train_test_split(
    X_dna_train_all, X_prot_train_all, X_num_train_all, y_train_all,
    test_size=0.30, random_state=42 # %70 Base Model Eğitimi, %30 Meta Model Eğitimi
)

if not os.path.exists('model_stack'): os.makedirs('model_stack')

# --- 5. MODEL EĞİTİMİ (STACKING BASE MODELS - GÜNCELLENMİŞ KISIM) ---

# --- MODEL 1: CNN (Derin Öğrenme) ---
print("\n>> [1/9] CNN (Deep Learning) Eğitiliyor...")
in_dna = Input(shape=(11,)); emb_dna = Embedding(len(dna_tok.word_index)+1, 12)(in_dna)
x1 = GlobalMaxPooling1D()(Conv1D(64, 3, activation='relu')(emb_dna))
in_prot = Input(shape=(11,)); emb_prot = Embedding(len(prot_tok.word_index)+1, 12)(in_prot)
x2 = GlobalMaxPooling1D()(Conv1D(64, 3, activation='relu')(emb_prot))
in_num = Input(shape=(6,)); x3 = BatchNormalization()(Dense(32, activation='relu')(in_num))
merged = Concatenate()([x1, x2, x3])
out = Dense(1, activation='sigmoid')(Dropout(0.3)(Dense(32, activation='relu')(merged)))

model_cnn = Model(inputs=[in_dna, in_prot, in_num], outputs=out)
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

class BestEpoch(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 10 == 0: print(f"    - Epoch {epoch} bitti. Loss: {logs['loss']:.4f}")

# CNN için class weight hesaplama (Manuel)
cw = {0: 1.0, 1: (np.sum(y_base==0) / np.sum(y_base==1))} if np.sum(y_base==1) > 0 else {0:1.0, 1:1.0}
model_cnn.fit([X_dna_base, X_prot_base, X_num_base], y_base, epochs=30, batch_size=32, verbose=0, callbacks=[BestEpoch()], class_weight=cw)
model_cnn.save('model_stack/model_cnn.h5')

# --- DİĞER MODELLER (Class Weight Balanced Eklendi) ---
print(">> [2/9] XGBoost Eğitiliyor... (Scale Pos Weight Aktif)")
# XGBoost için scale_pos_weight hesaplama
neg_count = np.sum(y_base == 0)
pos_count = np.sum(y_base == 1)
spw = neg_count / pos_count if pos_count > 0 else 1.0
model_xgb = xgb.XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.05, scale_pos_weight=spw).fit(X_num_base, y_base)

print(">> [3/9] LightGBM Eğitiliyor... (Balanced)")
model_lgbm = lgb.LGBMClassifier(n_estimators=200, class_weight='balanced', verbose=-1).fit(X_num_base, y_base)

print(">> [4/9] Random Forest Eğitiliyor... (Balanced)")
model_rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42).fit(X_num_base, y_base)

print(">> [5/9] Decision Tree Eğitiliyor... (Balanced)")
model_dt = DecisionTreeClassifier(class_weight='balanced', random_state=42).fit(X_num_base, y_base)

print(">> [6/9] SVM Eğitiliyor... (Balanced)")
model_svm = SVC(probability=True, class_weight='balanced', random_state=42).fit(X_num_base, y_base)

print(">> [7/9] KNN Eğitiliyor... (Distance Weighting)")
model_knn = KNeighborsClassifier(n_neighbors=5, weights='distance').fit(X_num_base, y_base)

print(">> [8/9] Naive Bayes Eğitiliyor...")
model_nb = GaussianNB().fit(X_num_base, y_base)

print(">> [9/9] Lojistik Regresyon (Base) Eğitiliyor... (Balanced)")
model_lr_base = LogisticRegression(max_iter=2000, class_weight='balanced').fit(X_num_base, y_base)

# --- 6. META MODEL (STACKING) EĞİTİMİ ---
print("\n>> [META] Tahminler Toplanıyor (Stacking)...")

# Meta seti üzerinde tahminler al
p_cnn = model_cnn.predict([X_dna_meta, X_prot_meta, X_num_meta], verbose=0).flatten()
p_xgb = model_xgb.predict_proba(X_num_meta)[:, 1]
p_lgbm = model_lgbm.predict_proba(X_num_meta)[:, 1]
p_rf = model_rf.predict_proba(X_num_meta)[:, 1]
p_dt = model_dt.predict_proba(X_num_meta)[:, 1]
p_svm = model_svm.predict_proba(X_num_meta)[:, 1]
p_knn = model_knn.predict_proba(X_num_meta)[:, 1]
p_nb = model_nb.predict_proba(X_num_meta)[:, 1]
p_lr = model_lr_base.predict_proba(X_num_meta)[:, 1]

# Stack matrisini oluştur
stack_X_meta = np.column_stack((p_cnn, p_xgb, p_lgbm, p_rf, p_dt, p_svm, p_knn, p_nb, p_lr))

# Meta Modeli Eğit
print(">> Meta Model (Hakem) eğitiliyor...")
meta_model = LogisticRegression(class_weight='balanced') # Meta modele de balanced dedik
meta_model.fit(stack_X_meta, y_meta)

# --- 7. FİNAL TEST (test.txt İLE) ---
if X_dna_test is not None:
    print("\n" + "="*50)
    print(f"🏆 FİNAL TEST SONUÇLARI (Test.txt Verisi Üzerinde)")

    # 1. Base modeller test verisini tahmin eder
    t_cnn = model_cnn.predict([X_dna_test, X_prot_test, X_num_test], verbose=0).flatten()
    t_xgb = model_xgb.predict_proba(X_num_test)[:, 1]
    t_lgbm = model_lgbm.predict_proba(X_num_test)[:, 1]
    t_rf = model_rf.predict_proba(X_num_test)[:, 1]
    t_dt = model_dt.predict_proba(X_num_test)[:, 1]
    t_svm = model_svm.predict_proba(X_num_test)[:, 1]
    t_knn = model_knn.predict_proba(X_num_test)[:, 1]
    t_nb = model_nb.predict_proba(X_num_test)[:, 1]
    t_lr = model_lr_base.predict_proba(X_num_test)[:, 1]

    # 2. Stack matrisi oluşturulur
    stack_X_final = np.column_stack((t_cnn, t_xgb, t_lgbm, t_rf, t_dt, t_svm, t_knn, t_nb, t_lr))

    # 3. Meta model son kararı verir
    final_preds = meta_model.predict(stack_X_final)

    # 4. Metrikler
    final_acc = accuracy_score(y_test, final_preds)
    final_f1 = f1_score(y_test, final_preds)

    print(f"✅ Accuracy : %{final_acc*100:.2f}")
    print(f"🚀 F1 Score : %{final_f1*100:.2f}")
    print("-" * 50)
    print("Detaylı Rapor:")
    print(classification_report(y_test, final_preds, target_names=["Negatif", "Pozitif"]))
    print("="*50)

else:
    print("\n⚠️ Test dosyası olmadığı için final testi atlandı.")

# --- KAYDETME İŞLEMLERİ ---
print(">> Modeller 'model_stack' klasörüne kaydediliyor...")
joblib.dump(meta_model, 'model_stack/meta_model.pkl')
joblib.dump(model_xgb, 'model_stack/xgb.pkl')
joblib.dump(model_lgbm, 'model_stack/lgbm.pkl')
joblib.dump(model_rf, 'model_stack/rf.pkl')
joblib.dump(model_dt, 'model_stack/dt.pkl')
joblib.dump(model_svm, 'model_stack/svm.pkl')
joblib.dump(model_knn, 'model_stack/knn.pkl')
joblib.dump(model_nb, 'model_stack/nb.pkl')
joblib.dump(model_lr_base, 'model_stack/lr_base.pkl')

with open('model_stack/dna_tok.pickle', 'wb') as f: pickle.dump(dna_tok, f)
with open('model_stack/prot_tok.pickle', 'wb') as f: pickle.dump(prot_tok, f)

print("✅ HER ŞEY HAZIR! EĞİTİM TAMAMLANDI.")

>> SİSTEM BAŞLATILIYOR (GÜNCELLENMİŞ EĞİTİM MODU)...

--- EĞİTİM VERİSİ YÜKLENİYOR ---
>> 'veri.txt' dosyası okunuyor...
✅ 'veri.txt' başarıyla yüklendi. Toplam kayıt: 100000

--- TEST VERİSİ YÜKLENİYOR ---
>> 'test.txt' dosyası okunuyor...
✅ 'test.txt' başarıyla yüklendi. Toplam kayıt: 10000

>> Tokenizer eğitiliyor...

>> Eğitim verisi 'Base' ve 'Meta' eğitim setlerine ayrılıyor...

>> [1/9] CNN (Deep Learning) Eğitiliyor...
    - Epoch 0 bitti. Loss: 0.0319
    - Epoch 10 bitti. Loss: 0.0082
    - Epoch 20 bitti. Loss: 0.0050


>> [2/9] XGBoost Eğitiliyor... (Scale Pos Weight Aktif)
>> [3/9] LightGBM Eğitiliyor... (Balanced)
>> [4/9] Random Forest Eğitiliyor... (Balanced)
>> [5/9] Decision Tree Eğitiliyor... (Balanced)
>> [6/9] SVM Eğitiliyor... (Balanced)
>> [7/9] KNN Eğitiliyor... (Distance Weighting)
>> [8/9] Naive Bayes Eğitiliyor...
>> [9/9] Lojistik Regresyon (Base) Eğitiliyor... (Balanced)

>> [META] Tahminler Toplanıyor (Stacking)...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


>> Meta Model (Hakem) eğitiliyor...

🏆 FİNAL TEST SONUÇLARI (Test.txt Verisi Üzerinde)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ Accuracy : %99.87
🚀 F1 Score : %99.89
--------------------------------------------------
Detaylı Rapor:
              precision    recall  f1-score   support

     Negatif       1.00      1.00      1.00      4290
     Pozitif       1.00      1.00      1.00      5710

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000

>> Modeller 'model_stack' klasörüne kaydediliyor...
✅ HER ŞEY HAZIR! EĞİTİM TAMAMLANDI.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [23]:
# DOSYA ADI: canli_test_zorlu.py
import numpy as np
import pickle
import joblib
import time
import os
import random
import tensorflow as tf
import warnings
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix, classification_report

# --- EKRAN TEMİZLİĞİ ---
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Renkler
YESIL = "\033[92m"
KIRMIZI = "\033[91m"
MAVI = "\033[94m"
SARI = "\033[93m"
CYAN = "\033[96m"
MOR = "\033[95m"
RESET = "\033[0m"

print(f"{SARI}>> CANLI TEST BAŞLIYOR (DENGELİ MOD & EZBER KONTROLÜ)...{RESET}")

# --- 1. MODELLERİ YÜKLE ---
try:
    with open('model_stack/dna_tok.pickle', 'rb') as f: dna_tok = pickle.load(f)
    with open('model_stack/prot_tok.pickle', 'rb') as f: prot_tok = pickle.load(f)
    model_cnn = load_model('model_stack/model_cnn.h5')
    model_xgb = joblib.load('model_stack/xgb.pkl')
    model_lgbm = joblib.load('model_stack/lgbm.pkl')
    model_rf = joblib.load('model_stack/rf.pkl')
    model_dt = joblib.load('model_stack/dt.pkl')
    model_svm = joblib.load('model_stack/svm.pkl')
    model_knn = joblib.load('model_stack/knn.pkl')
    model_nb = joblib.load('model_stack/nb.pkl')
    model_lr_base = joblib.load('model_stack/lr_base.pkl')
    meta_model = joblib.load('model_stack/meta_model.pkl')
    print(f"{YESIL}>> Modeller yüklendi.{RESET}")
except Exception as e:
    print(f"{KIRMIZI}❌ HATA: Modeller yüklenemedi!{RESET}")
    exit()

# --- 2. YARDIMCI FONKSİYONLAR ---
def generate_sequence_with_pattern(length, vocab, pattern=None):
    seq = list(np.random.choice(list(vocab), size=length))
    if pattern:
        start = np.random.randint(0, length - len(pattern) + 1)
        for i, char in enumerate(pattern):
            seq[start + i] = char
    return "".join(seq)

# --- 3. TEST VERİSİ ÜRETİCİ (GÜNCELLENDİ) ---
def canli_veri_uret(zorla_etiket=None):
    if zorla_etiket is None:
        hedef = 1 if random.random() > 0.5 else 0
    else:
        hedef = zorla_etiket

    if hedef == 1: # HASTA
        risk = np.random.uniform(0.55, 0.99)
        cons = np.random.uniform(6.0, 10.0)
        maf = np.random.uniform(0.00, 0.15)
        hydro = np.random.uniform(-5.0, -1.0)
        polar = np.random.uniform(2.0, 5.0)
        weight_base = np.random.uniform(50.0, 90.0)
    else: # SAĞLAM
        risk = np.random.uniform(0.00, 0.50)
        cons = np.random.uniform(0.0, 5.0)
        maf = np.random.uniform(0.20, 0.50)
        hydro = np.random.uniform(-1.0, 5.0)
        polar = np.random.uniform(-5.0, 1.0)
        weight_base = np.random.uniform(10.0, 50.0)

    score = (risk * 0.50) + (cons/10 * 0.30) + ((0.5 - maf) * 0.1) + (abs(hydro)/5 * 0.1)
    score += np.random.uniform(-0.03, 0.03) # Gürültü

    gercek_etiket = 1 if score > 0.60 else 0

    # Motif Ekleme
    if gercek_etiket == 1:
        dna = generate_sequence_with_pattern(11, "ACGT", pattern="CGT") if random.random() > 0.2 else generate_sequence_with_pattern(11, "ACGT")
        prot = generate_sequence_with_pattern(11, "ACDEFGHIKLMNPQRSTVWY", pattern="WW") if random.random() > 0.2 else generate_sequence_with_pattern(11, "ACDEFGHIKLMNPQRSTVWY")
        weight = weight_base + 10
    else:
        dna = generate_sequence_with_pattern(11, "ACGT", pattern="AAA") if random.random() > 0.8 else generate_sequence_with_pattern(11, "ACGT")
        prot = generate_sequence_with_pattern(11, "ACDEFGHIKLMNPQRSTVWY")
        weight = weight_base - 10

    return dna, prot, [risk, maf, cons, hydro, polar, weight], gercek_etiket

# --- 4. TAHMİN MEKANİZMASI ---
def tahmin_et(dna, prot, feats):
    s_d = pad_sequences(dna_tok.texts_to_sequences([dna]), maxlen=11, padding='post')
    s_p = pad_sequences(prot_tok.texts_to_sequences([prot]), maxlen=11, padding='post')
    f_n = np.array([feats]).astype('float32')

    p_cnn = model_cnn.predict([s_d, s_p, f_n], verbose=0).flatten()[0]
    p_xgb = model_xgb.predict_proba(f_n)[:, 1][0]
    p_lgbm = model_lgbm.predict_proba(f_n)[:, 1][0]
    p_rf = model_rf.predict_proba(f_n)[:, 1][0]
    p_dt = model_dt.predict_proba(f_n)[:, 1][0]
    p_svm = model_svm.predict_proba(f_n)[:, 1][0]
    p_knn = model_knn.predict_proba(f_n)[:, 1][0]
    p_nb = model_nb.predict_proba(f_n)[:, 1][0]
    p_lr = model_lr_base.predict_proba(f_n)[:, 1][0]

    stack_input = np.array([[p_cnn, p_xgb, p_lgbm, p_rf, p_dt, p_svm, p_knn, p_nb, p_lr]])
    final_pred = meta_model.predict(stack_input)[0]

    return final_pred

# --- 5. SİMÜLASYON ---
y_true, y_pred = [], []
TOPLAM_TEST = 1000
YARI_TEST = TOPLAM_TEST // 2

print(f"\n{'TÜR':<10} | {'DNA':<12} | {'RİSK':<5} | {'GERÇEK':<8} | {'TAHMİN':<8} | {'SONUÇ'}")
print("-" * 75)

for i in range(TOPLAM_TEST):
    hedef = 1 if i < YARI_TEST else 0
    dna, prot, feats, gercek = canli_veri_uret(zorla_etiket=hedef)
    pred = tahmin_et(dna, prot, feats)

    y_true.append(gercek)
    y_pred.append(pred)

    if i < 3 or (i > YARI_TEST-2 and i < YARI_TEST+2) or i > TOPLAM_TEST-3:
        str_gercek = "HASTA" if gercek == 1 else "SAĞLAM"
        str_tahmin = "HASTA" if pred == 1 else "SAĞLAM"
        ikon = f"{YESIL}✅{RESET}" if gercek == pred else f"{KIRMIZI}❌{RESET}"
        print(f"{' [TEST] ': <10} | {dna[:8]:<12} | {feats[0]:.2f}  | {str_gercek:<8} | {str_tahmin:<8} | {ikon}")

# --- 6. RAPORLAMA VE EZBER ANALİZİ ---
acc = accuracy_score(y_true, y_pred)
kappa = cohen_kappa_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print("\n" + "="*50)
print(f"📊 FİNAL PERFORMANS VE EZBER RAPORU")
print("="*50)
print(f"✅ Doğruluk (Accuracy) : %{acc*100:.2f}")
print(f"🧠 Cohen's Kappa       : {kappa:.4f}")

print("\n🔍 EZBER ANALİZİ (MEMORIZATION CHECK):")
print("-" * 50)
if kappa > 0.80:
    print(f"{YESIL}🌟 MÜKEMMEL! Model EZBER YAPMAMIŞ.{RESET}")
    print(f"   Model mantığı kavramış ve hiç görmediği verileri çözebiliyor.")
    print(f"   (Kappa 0.80 üzeri 'Neredeyse Kusursuz' uyum demektir)")
elif kappa > 0.60:
    print(f"{SARI}⚠️ İYİ AMA RİSKLİ. Biraz ezber olabilir.{RESET}")
elif kappa < 0.40:
    print(f"{KIRMIZI}❌ KÖTÜ. Model muhtemelen şans eseri tutturuyor.{RESET}")

print("-" * 50)
tn, fp, fn, tp = cm.ravel()
print(f"🧩 KARMAŞIKLIK MATRİSİ:")
print(f"   Gerçek SAĞLAM: {tn} | Yanlış Alarm: {fp}")
print(f"   Gerçek HASTA : {tp} | Kaçırılan   : {fn}")

>> CANLI TEST BAŞLIYOR (DENGELİ MOD & EZBER KONTROLÜ)...
>> Modeller yüklendi.

TÜR        | DNA          | RİSK  | GERÇEK   | TAHMİN   | SONUÇ
---------------------------------------------------------------------------
 [TEST]    | TGATTTCG     | 0.85  | HASTA    | HASTA    | ✅
 [TEST]    | CCTCGTCT     | 0.91  | HASTA    | HASTA    | ✅
 [TEST]    | GCGTCAAC     | 0.84  | HASTA    | HASTA    | ✅
 [TEST]    | ACCCCGTT     | 0.96  | HASTA    | HASTA    | ✅
 [TEST]    | TCAACACC     | 0.36  | SAĞLAM   | SAĞLAM   | ✅
 [TEST]    | TGTTGGTG     | 0.25  | SAĞLAM   | SAĞLAM   | ✅
 [TEST]    | CTATCTCT     | 0.04  | SAĞLAM   | SAĞLAM   | ✅
 [TEST]    | TATCCCAA     | 0.05  | SAĞLAM   | SAĞLAM   | ✅

📊 FİNAL PERFORMANS VE EZBER RAPORU
✅ Doğruluk (Accuracy) : %99.70
🧠 Cohen's Kappa       : 0.9940

🔍 EZBER ANALİZİ (MEMORIZATION CHECK):
--------------------------------------------------
🌟 MÜKEMMEL! Model EZBER YAPMAMIŞ.
   Model mantığı kavramış ve hiç görmediği verileri çözebiliyor.
   (Kappa 0.8